## Definição do Problema de Negócio

In [ ]:
# 1 - Estrategia da Solução
-- Entender o objetivo do projeto
    1.1 - Identificar qual o melhor canal do YouTube para realizar anuncio, a fim de obter uma maior exposição da marca anunciada.
    1.2 - Identificar se há um dia da semana o que seja melhor para anunciar?
    
-- Proposta de solução
    1.3 - Tabela - vídeo | Visualizações por categoria (Exploratory Data Analysis)
    1.4 - Tabela - vídeo | Visualizações por canal (Exploratory Data Analysis)
    1.5 - Tabela - vídeo | Visualizações por dia (Exploratory Data Analysis)
    1.6 - Tabela - vídeo | Atributos | Correlação ()
    1.7 - Tabela - vídeo | O vídeo é tendência (0 , 1)
    1.8 - Tabela - vídeo | Realizar analise de Sentimento nos Comentários
    1.9 - Tabela - vídeo | Realizar Processamento de Linguaguem Natural
    
    
# 2 - Ciclo para criar solução
    
    2.1 - Pipeline de Dados funcional de ponta a ponta (coleta de dados até treinamento do modelo)
        - 
    
    2.2 - Entender os Dados -> (Inconsistência nos dados) Análise da Estatística Descritiva 
        média, mediana, std, min, max, kurtoses, skew - Profilling
        
    2.3 - Feature Engenieering -> Criar variáveis que modelem o fenomeno
            mês|dia da semana|%_dislikes|tempo que vira tendência|
    
    2.4 - Criar a Hipotese do Negócio - (sempre deve ser uma afirmação)
        - H1: Os dia da semana tem relação com a quantidade de visualizações.
        - H2: Há uma diferença muito grande no nº de vizualizações entre os canais
        - H3: Alguns canais tem um aumento de vizualizações em determinados dias da semana
            
    2.5 - Análise Exploratória dos dados
        - Validações ou Refutações das Hipóteses
        
    2.6 - Treinar o modelo
        - By the book

# 2 - Tipos de Códigos

## Imports

In [1]:
# import dos módulos
import locale
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pandas import read_csv, read_excel
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import glob
import os, json

# Normaliza os dados JSON semiestruturados em uma tabela plana.
from pandas.io.json import json_normalize  

%matplotlib inline
mpl.__version__

'3.0.2'

In [2]:
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

'pt_BR.utf8'

In [3]:
# Utiliza os modulos do diretório lib
import sys
sys.path.append("lib")
import geradados_youtube

## Loading Data

### Arquivo csv

In [4]:
# Leitura e Concatenação dos arquivos *.csv do diretório 
geradados_youtube.get_concat_save_data()

In [57]:
# Leitura do arquivo concatenado
arquivo = "Dados/merged_files.csv"
dadosCsv = read_csv(arquivo, encoding = "latin-1")
print(dadosCsv.shape)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 4


In [55]:
dadosCsv.head(5)

,Unnamed: 0,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. BeyoncÃÂ©,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. BeyoncÃÂ...
1,1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...
2,2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO Ã¢ÂÂ¶ \n\nSUBSCRIBE Ã...
3,3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
4,4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,Ã°ÂÂÂ§: https://ad.gt/yt-perfect\nÃ°ÂÂÂ°: ...


### Arquivo js

In [7]:
#  Leitura e Concatenação dos arquivos *.json do diretório 
contents = []
json_dir_name = "arquivos/"

json_pattern = os.path.join(json_dir_name,'*.json')
file_list = glob.glob(json_pattern)
for file in file_list:
    contents.append(pd.read_json(file))
    dadosJs = pd.concat(contents)

In [8]:
dadosJs.head(5)

,kind,etag,items
0,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."
1,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."
2,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."
3,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."
4,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."


## Data Preparation

In [9]:
# Fazendo um subset nos dados, pegando apenas as informações da coluna 'items'
df_djson = dadosJs['items']

In [10]:
df_djson.head(5)

0    {'kind': 'youtube#videoCategory', 'etag': '"ld...
1    {'kind': 'youtube#videoCategory', 'etag': '"ld...
2    {'kind': 'youtube#videoCategory', 'etag': '"ld...
3    {'kind': 'youtube#videoCategory', 'etag': '"ld...
4    {'kind': 'youtube#videoCategory', 'etag': '"ld...
Name: items, dtype: object

In [11]:
# Aplicando estrutura no dados
dadosJs_norm = pd.json_normalize(df_djson)

In [12]:
dadosJs_norm.head(5)

,kind,etag,id,snippet.channelId,snippet.title,snippet.assignable
0,youtube#videoCategory,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/Xy1mB4_yLrHy_BmKm...",1,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True
1,youtube#videoCategory,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/UZ1oLIIz2dxIhO45Z...",2,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles,True
2,youtube#videoCategory,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/nqRIq97-xe5XRZTxb...",10,UCBR8-60-B28hp2BmDPdntcQ,Music,True
3,youtube#videoCategory,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/HwXKamM1Q20q9BN-o...",15,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals,True
4,youtube#videoCategory,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/9GQMSRjrZdHeb1OEM...",17,UCBR8-60-B28hp2BmDPdntcQ,Sports,True


In [13]:
# Fazendo um subset nos dados, pegando apenas as colunas 'id','snippet.title'.
Js_category = dadosJs_norm[['id','snippet.title']]

In [14]:
# Removendo as informações duplicadas
Js_category = Js_category.drop_duplicates()

In [15]:
Js_category.nunique()

id               32
snippet.title    31
dtype: int64

In [16]:
Js_category

,id,snippet.title
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports
5,18,Short Movies
6,19,Travel & Events
7,20,Gaming
8,21,Videoblogging
9,22,People & Blogs


In [17]:
# Obtendo as informações do DataFrame
Js_category.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 295
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             32 non-null     object
 1   snippet.title  32 non-null     object
dtypes: object(2)
memory usage: 768.0+ bytes


In [18]:
# Alteração do tipo da coluna 'id' de string para inteiro
Js_category.id = Js_category.id .astype(np.int64)

In [19]:
# Ordenando o DataFrame e colocando a coluna id como index
Js_category.sort_values('id', ascending=True).set_index('id')

,snippet.title
id,
1,Film & Animation
2,Autos & Vehicles
10,Music
15,Pets & Animals
17,Sports
18,Short Movies
19,Travel & Events
20,Gaming
21,Videoblogging


In [20]:
# Obtendo as informações do DataFrame
Js_category.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 295
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             32 non-null     int64 
 1   snippet.title  32 non-null     object
dtypes: int64(1), object(1)
memory usage: 768.0+ bytes


In [21]:
Js_category = Js_category.rename(columns={"id":"category_id", "snippet.title":"snippet_title"})

In [22]:
Js_category.head(5)

,category_id,snippet_title
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


In [23]:
# Salvando o arquivo final "category.csv" no diretório:
Js_category.to_csv("Dados/category.csv", index=False)

### Importing files and joining two DataFrames

In [26]:
# Leitura do arquivo .csv do diretório 
arquivo = "Dados/category.csv"
df_category = read_csv(arquivo, encoding ='utf-8')
print(df_category.shape)

(32, 2)


In [36]:
# Unindo o dataFrame pela coluna 'category_id' para obter o título da categoria
df_dados = pd.merge(dadosCsv,df_category, how='left', on='category_id')

In [37]:
# Obtendo os nomes das colunas
df_dados.columns

Index(['Unnamed: 0', 'video_id', 'trending_date', 'title', 'channel_title',
       'category_id', 'publish_time', 'tags', 'views', 'likes', 'dislikes',
       'comment_count', 'thumbnail_link', 'comments_disabled',
       'ratings_disabled', 'video_error_or_removed', 'description',
       'snippet_title'],
      dtype='object')

In [38]:
df_dados = df_dados.drop(['Unnamed: 0', 'thumbnail_link', 'comments_disabled','ratings_disabled', 'video_error_or_removed'], axis=1)

In [39]:
df_dados.shape

(334993, 14)

In [43]:
# Organizando as colunas do DataFrame de acordo com a lista
columnsTitles = ['video_id', 'category_id', 'snippet_title','channel_title',  'title', 'trending_date', 'publish_time', 'views', 'likes', 'dislikes', 'comment_count','tags', 'description']

df_dados = df_dados.reindex(columns=columnsTitles)

In [47]:
df_dados.head(2)

,video_id,category_id,snippet_title,channel_title,title,trending_date,publish_time,views,likes,dislikes,comment_count,tags,description
0,n1WpP7iowLc,10,Music,EminemVEVO,Eminem - Walk On Water (Audio) ft. BeyoncÃ©,17.14.11,2017-11-10T17:00:03.000Z,17158579,787425,43420,125882,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",Eminem's new track Walk on Water ft. BeyoncÃ© ...
1,0dBIkQ4Mz1M,23,Comedy,iDubbbzTV,PLUSH - Bad Unboxing Fan Mail,17.14.11,2017-11-13T17:00:00.000Z,1014651,127794,1688,13030,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",STill got a lot of packages. Probably will las...


## Feature Engenieering 

## Exploratory Data Analysis

In [46]:
df_dados['snippet_title'].value_counts()

Entertainment            99042
People & Blogs           50842
Music                    36042
News & Politics          34801
Comedy                   23513
Sports                   21510
Film & Animation         18587
Howto & Style            14710
Gaming                   10681
Education                 6132
Science & Technology      5770
Autos & Vehicles          4350
Pets & Animals            3943
Nonprofits & Activism     2738
Travel & Events           1374
Shows                      917
Movies                      36
Trailers                     5
Name: snippet_title, dtype: int64

## PLots

### Qual a categoria de vídeo tem maior visaulizações?

In [ ]:
# Agrupamentos
agg_views = (dados.groupby(["snippet_title"]).sum()["views"])/10000000
agg_likes = (dados.groupby(["snippet_title"]).sum()["likes"])/10000000
agg_dislikes = (dados.groupby(["snippet_title"]).sum()["dislikes"])/10000000
count_all = (dados.sum()["views"])/10000000
perc_views = (agg_views / count_all)*100

# Dataframe para os resultados
pd.options.display.float_format = '{:,.2f}'.format 
summary = pd.DataFrame({"Total_Views":agg_views,
                        "%_Views" : perc_views,
                        "Total_Likes":agg_likes,
                        "Total_Dislikes" : agg_dislikes})


# Data Munging
#summary = summary.round(2)
#summary["Total_Views"] = summary["Total_Views"].map("{:,.2f}".format)
#summary_format = summary.loc[:, ["Número Viagens","Número de diárias", "Valor Total"]]
sumary = summary.sort_values(by = 'Total_Views', ascending = False)
summary["%_Views"] = summary["%_Views"].map("{:,.2f}%".format)             

In [ ]:
sumary

In [ ]:
# Gráfico da uantidade de visializações por milhão
plt.figure(figsize=(20,20))
tot_view_gs = mpl.gridspec.GridSpec(1, 1)
fig, ax = plt.subplots()

quant = sumary['Total_Views']

nomes = [ 'Music','Entertainment','Comedy','People & Blogs', 'Sports','Film & Animation','News & Politics','Gaming','Howto & Style','Nonprofits & Activism','Science & Technology', 'Pets & Animals',
        'Autos & Vehicles', 'Education','Shows','Travel & Events']
colors = ['LightSlateGray']
ax.grid(color='#808080', linestyle='--', linewidth=0.5)
ax.barh(nomes, quant, color= colors)
ax.set_xlabel("", color = '#F5F5DC', fontsize=18)
ax.set_ylabel('Categorias', color = 'white', fontsize=12)
ax.set_title('Quantidade de visializações por milhão', fontsize=12, color = 'white')

plt.show()

### Qual categoria tem mais likes? 

In [ ]:
# Gráfico de Total de Likes - milhões
plt.figure(figsize=(20,20))
fig, ax = plt.subplots()

quant = sumary['Total_Likes']

nomes = [ 'Music','Entertainment','Comedy','People & Blogs', 'Sports','Film & Animation','News & Politics','Gaming','Howto & Style','Nonprofits & Activism','Science & Technology', 'Pets & Animals',
        'Autos & Vehicles', 'Education','Shows','Travel & Events']
colors = ['LightSlateGray']
ax.grid(color='#808080', linestyle='--', linewidth=0.5)
ax.barh(nomes, quant, color= colors)
ax.set_xlabel("", color = '#F5F5DC', fontsize=18)
ax.set_ylabel('Categorias', color = 'white', fontsize=12)
ax.set_title('Total de Likes - milhões', fontsize=12, color = 'white')

plt.show()

### Qual categoria tem mais dislikes? 

In [ ]:
# Gráfico da quantidade de deputados por região
plt.figure(figsize=(20,20))
fig, ax = plt.subplots()

quant = sumary['Total_Dislikes']

nomes = [ 'Music','Entertainment','Comedy','People & Blogs', 'Sports','Film & Animation','News & Politics','Gaming','Howto & Style','Nonprofits & Activism','Science & Technology', 'Pets & Animals',
        'Autos & Vehicles', 'Education','Shows','Travel & Events']
colors = ['LightSlateGray']
ax.grid(color='#808080', linestyle='--', linewidth=0.5)
ax.barh(nomes, quant, color= colors)
ax.set_xlabel("", color = '#F5F5DC', fontsize=18)
ax.set_ylabel('Categorias', color = 'white', fontsize=12)
ax.set_title('Total de Dislikes - milhões', fontsize=12, color = 'white')

plt.show()

### Qual o dia de maior vizualização?

In [ ]:
# Agrupamentos
day_views = (dados.groupby(["ind_day"]).sum()["views"])/10000000
day_likes = (dados.groupby(["ind_day"]).sum()["likes"])/10000000
day_dislikes = (dados.groupby(["ind_day"]).sum()["dislikes"])/10000000
day_better = dados.groupby(["ind_day"]).mean()["time_up"]
count_all = dados.sum()["views"]/10000000
perc_days_views = (day_views / count_all)*100

# Dataframe para os resultados
pd.options.display.float_format = '{:,.1f}'.format 
summary_days = pd.DataFrame({"DaysViews": day_views,
                             "DaysLikes": day_likes,
                             "DaysDislikes": day_dislikes,
                             "DayBetter": day_better,
                             "%DayViews" : perc_days_views})


# Data Munging
summary = summary.round(2)
#summary_days["%DayViews"] = summary_days["%DayViews"].map("{:,.2f}".format)
#summary_format = summary.loc[:, ["Número Viagens","Número de diárias", "Valor Total"]]
summary_days = summary_days.sort_values(by = 'ind_day', ascending = True)
summary_days 

In [ ]:
summary_days.reset_index()

In [ ]:
fig, ax = plt.subplots()
plt.figure(figsize = (12,8))

quant = summary_days['DaysViews']
nomes = ['Segunda','Terça','Quarta','Quinta', 'Sexta', 'Sábado', 'Domingo']
colors = ['LightSlateGray']
ax.grid(color='#D3D3D3', linestyle='--', linewidth=0.2)
ax.bar(nomes,quant, color= colors)
ax.set_ylabel('Por milhão', color = 'white', fontsize=10)
ax.set_title('Total de Visializações - milhões/dia', fontsize=12, color = 'white')

In [ ]:
fig, ax = plt.subplots()
plt.figure(figsize = (12,8))

quant = summary_days['DaysLikes']
nomes = ['Segunda','Terça','Quarta','Quinta', 'Sexta', 'Sábado', 'Domingo']
colors = ['LightSlateGray']
ax.grid(color='#D3D3D3', linestyle='--', linewidth=0.2)
ax.bar(nomes,quant, color= colors)
ax.set_ylabel('Por milhão', color = 'white', fontsize=10)
ax.set_title('Total de Likes - milhões/dia', fontsize=12, color = 'white')

In [ ]:
fig, ax = plt.subplots()
plt.figure(figsize = (12,8))

quant = summary_days['DaysDislikes']
nomes = ['Segunda','Terça','Quarta','Quinta', 'Sexta', 'Sábado', 'Domingo']
colors = ['LightSlateGray']
ax.grid(color='#D3D3D3', linestyle='--', linewidth=0.2)
ax.bar(nomes,quant, color= colors)
ax.set_ylabel('Por milhão', color = 'white', fontsize=10)
ax.set_title('Total de Dislikes - milhões/dia', fontsize=12, color = 'white')

### Em média quantos dias um vídeo vira tendência?

In [ ]:
fig, ax = plt.subplots()
plt.figure(figsize = (12,4))

quant = summary_days['DayBetter']
nomes = ['Segunda','Terça','Quarta','Quinta', 'Sexta', 'Sábado', 'Domingo']
colors = ['LightSlateGray']
ax.grid(color='#D3D3D3', linestyle='--', linewidth=0.5)
ax.bar(nomes,quant, color= colors)
ax.set_ylabel('Quantidade', color='white')
ax.set_title('Média de dias que o vídeo viraliza', color='white')

plt.show()

### Em média quantos dias um vídeo vira tendência?

In [ ]:
pd.options.display.float_format = '{:,.1f}'.format 
day_mean = dados.groupby(["snippet_title"]).mean()["time_up"]
dayUp_mean = pd.DataFrame({"MeanDayUp": day_mean})
dayUp_mean = dayUp_mean.sort_values(by = 'MeanDayUp', ascending = True)
dayUp_mean

In [ ]:
Qual o dia de maior vizualização?
O dia em que o vídeo a publicado influência para que o vídeo vire tendência mais rápido?
Em média quantos dias um vídeo vira tendência?
Em média qual categoria tem mais likes e dislikes? 

In [ ]:
#.sort_values(by = option.value, ascending = False)
fig, ax = plt.subplots()

num_bins = 100

dayUp_mean.plot.kde(ax=ax, legend=False,color='red', title='Histogram:')

bins = dayUp_mean.plot.hist(num_bins, density=True, ax=ax)
ax.set_ylabel('MeanDayUp',color='#D3D3D3')
ax.grid(axis='MeanDayUp',color='#D3D3D3')
ax.set_facecolor(color='#D3D3D3')

# Tweak spacing to prevent clipping of ylabel
fig.tight_layout()
plt.show()

### correlação

In [ ]:
dados2 = dados.set_index('publish_date')
dados2.head(2)

In [ ]:
sns.heatmap(dados.isnull(),
           yticklabels=True,
           cbar=False,
           cmap='viridis')

In [ ]:
# Visualizações por dia da semana
ax = sns.boxplot(x="name_day",
           y='views',
           data=dados,
           orient="h",
           order=["Dinner", "Lunch"],
           palette='winter')

In [ ]:
# Comparação de visualizações entre final de semana e dia da semana
ax = sns.boxplot(x="day_week",
           y='views',
           data=dados,
           palette='winter')

In [ ]:
plt.figure(figsize = (14,6))

g = sns.boxplot(x='snippet_title', y='likes', data=dados, palette="Set1")
g.set_xticklabels(g.get_xticklabels(),rotation=45)
g.set_title("Distribuição de Likes por categoria ", fontsize=12)
g.set_xlabel("", fontsize=12)
g.set_ylabel("Likes(log)", fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize = (14,6))

g = sns.boxplot(x='name_day', y='dislikes', data=dados, palette="Set1")
g.set_xticklabels(g.get_xticklabels(),rotation=45)
g.set_title("Distribuição de Likes por dia da semana", fontsize=15)
g.set_xlabel("", fontsize=12)
g.set_ylabel("Likes(log)", fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize = (14,6))

g = sns.boxplot(x='day_week', y='time_up', data=dados, palette="Set1")
g.set_xticklabels(g.get_xticklabels(),rotation=45)
g.set_title("Distribuição de Likes por fim de semana/dia da semana", fontsize=15)
g.set_xlabel("", fontsize=12)
g.set_ylabel("time_up", fontsize=12)
plt.show()

In [ ]:
# Funções para desenhar modelos de regressão linear 
sns.set(color_codes=True)
sns.regplot(x="views", y="likes", data=dados);

In [ ]:

sns.set(style="white")

# Plot de likes por visualizações
sns.relplot(x="likes", y="views", hue="snippet_title", size="views",
            sizes=(40, 400), alpha=.5, palette="muted",
            height=6, data=dados)

### seaborn.boxplot

In [ ]:
sns.lmplot(x="views", y="likes", col="name_day", data=dados,
           col_wrap=2, height=5);

In [ ]:
# Use catplot()para combinar a boxplot()e a FacetGrid. Isso permite o agrupamento em variáveis categóricas adicionais. 
# Usar catplot()é mais seguro do que usar FacetGrid diretamente, pois garante a sincronização da ordem das variáveis entre as facetas:

g = sns.catplot(x="day_week", y="views",
                hue="likes", col="ind_day",
                data=dados, kind="box",
                height=4, aspect=.7);

In [ ]:
from scipy import stats
sns.distplot(dados.likes, fit = stats.norm);

### Times Series

In [ ]:
#https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas/
ts_dados = dados.iloc[:,[4,5,6,7,8,12,13,15,16]]

In [ ]:
# Salva o arquivo "csv" no diretório:
ts_dados.to_csv("C:/Users/lambe/OneDrive/Área de Trabalho/Projetos/MachineLearning/ts_dados.csv", index=False)

In [ ]:
# Lendo o arquivo .csv do diretório
arquivo = "C:/Users/lambe/OneDrive/Área de Trabalho/Projetos/MachineLearning/ts_dados.csv"
dados_TS = read_csv(arquivo, encoding ='utf-8', index_col=0, parse_dates=True)

In [ ]:
dados_TS.head(2)

In [ ]:
dados_TS.info()

In [ ]:
dados_TS.loc['2018-02-15']

In [ ]:
# define o tamanho de figura padrão
sns.set(rc={'figure.figsize':(15, 12)})

In [ ]:
ano_2018 = dados_TS.loc['2018-01-01':'2018-03-31']

In [ ]:
ano_2018['likes'].plot(linewidth=0.5);

In [ ]:
ax = dados_TS.loc['2018', 'likes'].plot()
ax.set_ylabel('Quant. likes');


In [ ]:
sns.boxplot(data=ano_2018, x='name_day', y='likes');

In [ ]:
times_sample = pd.date_range('2018-01-01', periods=90, freq='D')

In [ ]:
vld_sample = ano_2018.loc[times_sample, ['views','likes','dislikes']].copy()

In [ ]:
vld_sample

In [ ]:
agg_views_sample = vld_sample.groupby(["publish_date"]).sum()["views"]
agg_like_sample = vld_sample.groupby(["publish_date"]).sum()["likes"]
agg_dislike_sample = vld_sample.groupby(["publish_date"]).sum()["dislikes"]

In [ ]:
df_agg_sample = pd.DataFrame({"Total_Views": agg_views_sample,
                              "Total_Likes": agg_like_sample,
                              "Total_Dislikes": agg_dislike_sample})

In [ ]:
df_agg_sample.head()

In [ ]:
# Convert the data to daily frequency, without filling any missings
likes_freq = df_agg_sample.asfreq('D')
# Create a column with missings forward filled
#likes_freq['Total_Likes - Forward Fill'] = df_agg_sample.asfreq('D', method='ffill')
likes_freq

In [ ]:
likes_freq['Total_Views'].plot(linewidth=1);
likes_freq['Total_Likes'].plot(linewidth=0.8);
likes_freq['Total_Dislikes'].plot(linewidth=0.8);

In [ ]:
#likes_freq['Total_Views'].plot(linewidth=1);
#likes_freq['Total_Likes'].plot(linewidth=0.8);
#likes_freq['Total_Dislikes'].plot(linewidth=0.8);

ax1 = plt.subplot(212)
ax1.margins(0.05)           # Default margin is 0.05, value 0 means fit
likes_freq['Total_Views'].plot(linewidth=0.8);

ax2 = plt.subplot(221)
ax2.margins(0.05)           # Values >0.0 zoom out
likes_freq['Total_Likes'].plot(linewidth=0.8);
ax2.set_title('Zoomed out')

ax3 = plt.subplot(222)
ax3.margins(0.005)   # Values in (-0.5, 0.0) zooms in to center
likes_freq['Total_Dislikes'].plot(linewidth=0.8);
ax3.set_title('Zoomed in')

plt.show()

In [ ]:
# Using graph_objects 
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
freq = likes_freq.reset_index()

In [ ]:
freq.head(2)

In [ ]:
# Séries Temporais
fig  =  go.Figure ([ go.Scatter ( x = freq['publish_date'],  y = freq['Total_Likes'])]) 

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig . show ()

In [ ]:
fig  =  px.line ( freq  ,   x = 'publish_date' ,  y = 'Total_Views' ,  title = 'Série temporal com controle deslizante de intervalo e seletores' )

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [ ]:
#df  =  pd . read_csv ( 'https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv' )
fig  =  px.line ( freq  ,   x = 'publish_date' ,  y = 'Total_Views' ,  title = 'Série temporal com controle deslizante de intervalo e seletores' )

# Use string de data para definir o intervalo xeixo 
fig.update_layout ( xaxis_range = [ '2018-01-01' , '2018-03-31'  ], 
                  title_text = "Definir manualmente o intervalo de datas" ) 
fig.show ()

### Classificação do vídeo 

In [ ]:
# Criando uma coluna para classificar de acordo com as condições se o vídeo é trending
# Lista de condições
conditions = [
    (dados['views'] < 50000) | (dados['likes'] < 20000) | (dados['time_up'] > 5),
    (dados['views'] > 50000) & (dados['likes'] > 20000) & (dados['time_up'] <= 5)
    ]

# criando a lista de valores que representam cada condição
values = ['0', '1']

dados['is_trending'] = np.select(conditions, values)

### Grid

In [ ]:
def hide_axes(axes):
    axes.set_frame_on(False)
    [n.set_visible(False) for n in axes.get_xticklabels() + axes.get_yticklabels()]
    [n.set_visible(False) for n in axes.get_xticklines() + axes.get_yticklines()]


In [ ]:
def make_autos_price_plot(figure, gs = None, pddata = None, axes = None):
    if not axes:
        axes = figure.add_subplot(gs[0, 0])
    min_data = pddata.groupby("snippet_title", sort=True)["views"].min()
    max_data = pddata.groupby("snippet_title", sort=True)["views"].max()
    mean_data = pddata.groupby("snippet_title", sort=True)["views"].mean()
    make_ids = geradados.get_make_ids(pddata)
    axes.set_title("Auto Price Ranges", fontsize=20)
    axes.plot(make_ids, min_data, c=colors[2], linewidth=4, alpha=0.7)
    axes.plot(make_ids, mean_data, c=colors[3], linewidth=4, alpha=0.7)
    axes.plot(make_ids, max_data, c=colors[4], linewidth=4, alpha=0.7)
    axes.set_xticks(range(-1, 13))
    axes.set_xticklabels([" "] + geradados.get_make_labels(pddata))
    axes.set_xlabel("snippet_title", fontsize=16)
    axes.set_ylabel("views", fontsize=16)
    high_patch = mpl.patches.Patch(color=colors[4], alpha=0.7, label="High")
    mean_patch = mpl.patches.Patch(color=colors[3], alpha=0.7, label="Mean")
    low_patch = mpl.patches.Patch(color=colors[2], alpha=0.7, label="Low")
    axes.legend(handles=[high_patch, mean_patch, low_patch], loc=2)
    if gs:
        gs.tight_layout(figure)
    return axes

In [ ]:
#FIM
figure = plt.figure(figsize = (15, 5))
interaction = mpl.gridspec.GridSpec(1, 1)

prices_axes = interaction_plot(figure, interaction, dados)
plt.show()

In [ ]:
def hide_axes(axes):
    axes.set_frame_on(False)
    [n.set_visible(False) for n in axes.get_xticklabels() + axes.get_yticklabels()]
    [n.set_visible(False) for n in axes.get_xticklines() + axes.get_yticklines()]

In [ ]:
# Contruindo as camadas (sem dados)
figure = plt.figure(figsize=(10, 8))
gs_master = mpl.gridspec.GridSpec(4, 2, height_ratios=[1, 2, 8, 2])

# Camada 1 - Title
#gs_1 = mpl.gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=gs_master[0, :])
#title_axes = figure.add_subplot(gs_1[0])

gs_1 = mpl.gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = gs_master[0, :])
title_axes = figure.add_subplot(gs_1[0])
title_axes.set_title("Plots", fontsize = 30, color = "#cdced1")
fig  =  go.Figure ([ go.Scatter ( x = freq['publish_date'],  y = freq['Total_Likes'])]) 

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig . show ()

# Camada 2 - Price
gs_2 = mpl.gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=gs_master[1, :])
price_axes = figure.add_subplot(gs_2[0])

# Camada 3 - Risks & Radar
gs_31 = mpl.gridspec.GridSpecFromSubplotSpec(2, 2, height_ratios=[2, 1], subplot_spec=gs_master[2, :1])
risk_and_loss_axes = figure.add_subplot(gs_31[0, :])
risk_axes = figure.add_subplot(gs_31[1, :1])
loss_axes = figure.add_subplot(gs_31[1:, 1])
gs_32 = mpl.gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=gs_master[2, 1])
radar_axes = figure.add_subplot(gs_32[0])

# Camada 4 - MPG
gs_4 = mpl.gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=gs_master[3, :])
mpg_axes = figure.add_subplot(gs_4[0])

# Une as camas, ainda sem dados
gs_master.tight_layout(figure)
plt.show()
# Camada 1 - Title
gs_1 = mpl.gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=gs_master[0, :])
title_axes = figure.add_subplot(gs_1[0])

# Camada 2 - Price
gs_2 = mpl.gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=gs_master[1, :])
price_axes = figure.add_subplot(gs_2[0])

# Camada 3 - Risks & Radar
gs_31 = mpl.gridspec.GridSpecFromSubplotSpec(2, 2, height_ratios=[2, 1], subplot_spec=gs_master[2, :1])
risk_and_loss_axes = figure.add_subplot(gs_31[0, :])
risk_axes = figure.add_subplot(gs_31[1, :1])
loss_axes = figure.add_subplot(gs_31[1:, 1])
gs_32 = mpl.gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=gs_master[2, 1])
radar_axes = figure.add_subplot(gs_32[0])

# Camada 4 - MPG
gs_4 = mpl.gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=gs_master[3, :])
mpg_axes = figure.add_subplot(gs_4[0])

# Une as camas, ainda sem dados
gs_master.tight_layout(figure)
plt.show()

In [ ]:
# Construindo as camadas (com os dados)
figure = plt.figure(figsize = (15, 15))
gs_master = mpl.gridspec.GridSpec(4, 2, 
                                  height_ratios = [1, 24, 128, 32], 
                                  hspace = 0, 
                                  wspace = 0)

# Camada 1 - Title
gs_1 = mpl.gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = gs_master[0, :])
title_axes = figure.add_subplot(gs_1[0])
title_axes.set_title("Plots", fontsize = 30, color = "#cdced1")
geraplot.hide_axes(title_axes)


# Camada 2 - Price
gs_2 = mpl.gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = gs_master[1, :])
price_axes = figure.add_subplot(gs_2[0])
geraplot.make_autos_price_plot(figure, 
                               pddata = dados, 
                               axes = price_axes)


# Camada 3, Part I - Risks
gs_31 = mpl.gridspec.GridSpecFromSubplotSpec(2, 2, 
                                             height_ratios = [2, 1], 
                                             hspace = 0.4, 
                                             subplot_spec = gs_master[2, :1])
risk_and_loss_axes = figure.add_subplot(gs_31[0, :])
geraplot.make_autos_loss_and_risk_plot(figure, 
                                       pddata = dados_normalizados, 
                                       axes = risk_and_loss_axes, 
                                       x_label = False, 
                                       rotate_ticks = True)
risk_axes = figure.add_subplot(gs_31[1, :1])
geraplot.make_autos_riskiness_plot(figure, 
                                   pddata = dados_normalizados, 
                                   axes = risk_axes, 
                                   legend = False, 
                                   labels = False)
loss_axes = figure.add_subplot(gs_31[1:, 1])
geraplot.make_autos_losses_plot(figure, 
                                pddata = dados_normalizados, 
                                axes = loss_axes, 
                                legend = False, 
                                labels = False)


# Camada 3, Part II - Radar
gs_32 = mpl.gridspec.GridSpecFromSubplotSpec(5, 3, 
                                             height_ratios = [1, 20, 20, 20, 20], 
                                             hspace = 0.6, 
                                             wspace = 0, 
                                             subplot_spec = gs_master[2, 1])
(rows, cols) = geometry = gs_32.get_geometry()
title_axes = figure.add_subplot(gs_32[0, :])
inner_axes = []
projection = radar.RadarAxes(spoke_count = len(dados_normalizados.groupby("make").mean().columns))
[inner_axes.append(figure.add_subplot(m, projection = projection)) for m in [n for n in gs_32][cols:]]
geraplot.make_autos_radar_plot(figure, 
                               pddata = dados_normalizados, 
                               title_axes = title_axes, 
                               inner_axes = inner_axes, 
                               legend_axes = False, 
                               geometry = geometry)


# Camada 4 - MPG
gs_4 = mpl.gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = gs_master[3, :])
mpg_axes = figure.add_subplot(gs_4[0])
geraplot.make_autos_mpg_plot(figure, 
                             pddata = dados, 
                             axes = mpg_axes)

# Unindo as camadas
gs_master.tight_layout(figure)
plt.show()

In [ ]:
# Construindo as camadas (com os dados)
figure = plt.figure(figsize = (15, 15))
gs_master = mpl.gridspec.GridSpec(4, 2, 
                                  height_ratios = [1, 24, 128, 32], 
                                  hspace = 0, 
                                  wspace = 0)

# Camada 1 - Title
gs_1 = mpl.gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = gs_master[0, :])
title_axes = figure.add_subplot(gs_1[0])
title_axes.set_title("Plots", fontsize = 30, color = "#cdced1")
hide_axes(title_axes)


# Unindo as camadas
gs_master.tight_layout(figure)
plt.show()


In [ ]:
#teste

In [ ]:
### Color bars
#from bokeh.sampledata.autompg import autompg
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.transform import transform

color_mapper = LinearColorMapper(palette="Viridis256", low=data_low, high=data_high)
color_bar = ColorBar(color_mapper=color_mapper, location=(0,0))
p.add_layout(color_bar, 'right')


source = ColumnDataSource(dados)
color_mapper = LinearColorMapper(palette="Viridis256", low=dados.weight.min(), high=dados.weight.max())

p = figure(x_axis_label='Visualizações', y_axis_label='Likes', tools='', toolbar_location=None)
p.circle(x='views', y='likes', color=transform('weight', color_mapper), size=20, alpha=0.6, source=dados)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0), title='Weight')
p.add_layout(color_bar, 'right')

show(p)

In [ ]:
import pandas as pd

from bokeh.layouts import column, row
from bokeh.models import Select
from bokeh.palettes import Spectral5
from bokeh.plotting import curdoc, figure
from bokeh.sampledata.autompg import autompg_clean as df

df = df.copy()

SIZES = list(range(6, 22, 3))
COLORS = Spectral5
N_SIZES = len(SIZES)
N_COLORS = len(COLORS)

# data cleanup
df.cyl = df.cyl.astype(str)
df.yr = df.yr.astype(str)
del df['name']

columns = sorted(df.columns)
discrete = [x for x in columns if df[x].dtype == object]
continuous = [x for x in columns if x not in discrete]

def create_figure():
    xs = df[x.value].values
    ys = df[y.value].values
    x_title = x.value.title()
    y_title = y.value.title()

    kw = dict()
    if x.value in discrete:
        kw['x_range'] = sorted(set(xs))
    if y.value in discrete:
        kw['y_range'] = sorted(set(ys))
    kw['title'] = "%s vs %s" % (x_title, y_title)

    p = figure(plot_height=600, plot_width=800, tools='pan,box_zoom,hover,reset', **kw)
    p.xaxis.axis_label = x_title
    p.yaxis.axis_label = y_title

    if x.value in discrete:
        p.xaxis.major_label_orientation = pd.np.pi / 4

    sz = 9
    if size.value != 'None':
        if len(set(df[size.value])) > N_SIZES:
            groups = pd.qcut(df[size.value].values, N_SIZES, duplicates='drop')
        else:
            groups = pd.Categorical(df[size.value])
        sz = [SIZES[xx] for xx in groups.codes]

    c = "#31AADE"
    if color.value != 'None':
        if len(set(df[color.value])) > N_COLORS:
            groups = pd.qcut(df[color.value].values, N_COLORS, duplicates='drop')
        else:
            groups = pd.Categorical(df[color.value])
        c = [COLORS[xx] for xx in groups.codes]

    p.circle(x=xs, y=ys, color=c, size=sz, line_color="white", alpha=0.6, hover_color='white', hover_alpha=0.5)

    return p


def update(attr, old, new):
    layout.children[1] = create_figure()


x = Select(title='X-Axis', value='mpg', options=columns)
x.on_change('value', update)

y = Select(title='Y-Axis', value='hp', options=columns)
y.on_change('value', update)

size = Select(title='Size', value='None', options=['None'] + continuous)
size.on_change('value', update)

color = Select(title='Color', value='None', options=['None'] + continuous)
color.on_change('value', update)

controls = column(x, y, color, size, width=200)
layout = row(controls, create_figure())

curdoc().add_root(layout)
curdoc().title = "Crossfilter"

In [ ]:
crossfilter

### Links

In [ ]:
# Links
# https://www.vooo.pro/insights/guia-completo-para-criar-time-series-com-codigo-em-python/
# https://realpython.com/pandas-dataframe/#working-with-time-series
#https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas/
# https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/spectrum_demo.html#sphx-glr-gallery-lines-bars-and-markers-spectrum-demo-py
#https://preinventedwheel.com/easy-matplotlib-bar-chart/
#https://mybinder.org/v2/gh/bokeh/bokeh-notebooks/master?filepath=tutorial%2F00%20-%20Introduction%20and%20Setup.ipynb